In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

def explore_genetic_data(genome_base_path):
    """
    유전체 데이터(.bed, .fam, .bim)의 구조를 탐색합니다.
    
    Args:
        genome_base_path: 유전체 데이터 파일의 기본 경로 (확장자 제외)
    """
    print("\n" + "="*80)
    print("유전체 데이터 탐색 (.bed, .fam, .bim)")
    print("="*80)
    
    # 1. 파일 존재 확인
    extensions = ['.bed', '.fam', '.bim']
    for ext in extensions:
        file_path = f"{genome_base_path}{ext}"
        file_size = os.path.getsize(file_path) / (1024*1024) if os.path.exists(file_path) else 0
        print(f"{file_path} - {'존재함' if os.path.exists(file_path) else '존재하지 않음'} (크기: {file_size:.2f} MB)")
    
    # 2. .fam 파일 분석 (샘플 정보)
    fam_path = f"{genome_base_path}.fam"
    if os.path.exists(fam_path):
        try:
            # .fam 파일은 일반적으로 6개 열을 가짐: FID, IID, 부 ID, 모 ID, 성별, 표현형
            fam_df = pd.read_csv(fam_path, sep='\s+', header=None, 
                                names=['FID', 'IID', 'PID', 'MID', 'Sex', 'Phenotype'])
            
            print("\n.fam 파일 구조 (샘플 정보):")
            print(f"- 샘플 수: {len(fam_df)}")
            print(f"- 열 목록: {fam_df.columns.tolist()}")
            print("- 처음 5개 행:")
            print(fam_df.head())
            
            # 성별 분포
            if 'Sex' in fam_df.columns:
                sex_counts = fam_df['Sex'].value_counts()
                print("\n- 성별 분포:")
                for sex_code, count in sex_counts.items():
                    sex_label = "남성" if sex_code == 1 else "여성" if sex_code == 2 else "알 수 없음"
                    print(f"  {sex_label} (코드 {sex_code}): {count}명 ({count/len(fam_df)*100:.2f}%)")
        
        except Exception as e:
            print(f".fam 파일 분석 중 오류 발생: {e}")
    
    # 3. .bim 파일 분석 (SNP 정보)
    bim_path = f"{genome_base_path}.bim"
    if os.path.exists(bim_path):
        try:
            # .bim 파일은 일반적으로 6개 열을 가짐: 염색체, SNP ID, 유전적 거리, 위치, 대립유전자1, 대립유전자2
            bim_df = pd.read_csv(bim_path, sep='\s+', header=None,
                                names=['CHR', 'SNP', 'CM', 'POS', 'A1', 'A2'])
            
            print("\n.bim 파일 구조 (SNP 정보):")
            print(f"- SNP 수: {len(bim_df)}")
            print(f"- 열 목록: {bim_df.columns.tolist()}")
            print("- 처음 5개 행:")
            print(bim_df.head())
            
            # 염색체별 SNP 분포
            chr_counts = bim_df['CHR'].value_counts().sort_index()
            print("\n- 염색체별 SNP 수:")
            for chr_num, count in chr_counts.items():
                print(f"  염색체 {chr_num}: {count}개 SNP")
        
        except Exception as e:
            print(f".bim 파일 분석 중 오류 발생: {e}")
    
    # # 4. .bed 파일 분석 (유전자형 데이터)     # !주의: 파일이 너무 크기 때문에 분석에 너무 오래 걸릴 수 있음.
    # bed_path = f"{genome_base_path}.bed"
    # if os.path.exists(bed_path):
    #     try:
    #         print("\n.bed 파일 구조 (유전자형 데이터):")
            
    #         # pandas-plink 라이브러리가 필요함
    #         try:
    #             import pandas_plink
                
    #             # pandas-plink로 PLINK 파일 읽기
    #             print("pandas-plink 라이브러리를 사용하여 데이터 일부 읽기 중...")
    #             (bim, fam, G) = pandas_plink.read_plink(genome_base_path, verbose=False)
                
    #             # 기본 정보 출력
    #             print(f"- 유전자형 행렬 크기: {G.shape} (샘플 수 x SNP 수)")
                
    #             # 일부 샘플에 대한 정보 (처음 5개 SNP의 처음 5개 샘플)
    #             if G.shape[0] > 0 and G.shape[1] > 0:
    #                 print("\n- 유전자형 데이터 샘플 (처음 5개 샘플 x 처음 5개 SNP):")
    #                 sample_size = min(5, G.shape[0])
    #                 snp_size = min(5, G.shape[1])
                    
    #                 # 데이터의 일부만 추출
    #                 try:
    #                     G_sample = G[:sample_size, :snp_size].compute()
    #                     print(G_sample)
    #                 except:
    #                     print("데이터 일부 추출 시 오류 발생")
                
    #             # 결측값 비율
    #             try:
    #                 print("\n- 결측값 확인 중...")
    #                 # 작은 데이터셋에서만 이 계산을 수행 (큰 데이터셋에서는 메모리 문제 발생 가능)
    #                 if G.shape[0] * G.shape[1] < 1_000_000:  # 1백만 요소 이하인 경우
    #                     missing_rate = G.isnull().mean().compute()
    #                     print(f"  결측값 비율: {missing_rate:.4f}")
    #                 else:
    #                     print("  데이터셋이 너무 커서 결측값 계산을 건너뜁니다.")
    #             except:
    #                 print("  결측값 계산 중 오류 발생")
                
    #             # 대립유전자 빈도
    #             try:
    #                 print("\n- 대립유전자 빈도 계산 중...")
    #                 # 첫 10개 SNP의 대립유전자 빈도만 계산
    #                 n_snps_to_check = min(10, G.shape[1])
    #                 for i in range(n_snps_to_check):
    #                     snp_id = bim.iloc[i]['snp']
    #                     # 0은 동형접합 주요 대립유전자, 1은 이형접합, 2는 동형접합 부차 대립유전자
    #                     # 결측값은 제외하고 계산
    #                     try:
    #                         g = G[:, i].dropna().compute()
    #                         genotype_counts = np.bincount(g.astype(int), minlength=3)
    #                         total = genotype_counts.sum()
                            
    #                         if total > 0:
    #                             # 대립유전자 빈도 계산 (AF = (2*#2 + #1) / (2*total))
    #                             af = (2 * genotype_counts[2] + genotype_counts[1]) / (2 * total)
    #                             print(f"  SNP {snp_id}: AF = {af:.4f} (0/0: {genotype_counts[0]}, 0/1: {genotype_counts[1]}, 1/1: {genotype_counts[2]})")
    #                     except:
    #                         print(f"  SNP {snp_id} 대립유전자 빈도 계산 중 오류 발생")
    #             except:
    #                 print("  대립유전자 빈도 계산 중 오류 발생")
                    
    #         except ImportError:
    #             print("pandas-plink 라이브러리가 설치되어 있지 않습니다. 다음 명령어로 설치할 수 있습니다:")
    #             print("pip install pandas-plink")
    #             print("\n.bed 파일은 바이너리 형식이므로 pandas-plink 없이는 분석할 수 없습니다.")
    #             print("기본 정보만 제공합니다:")
                
    #             # 기본 정보만 제공
    #             with open(bed_path, 'rb') as f:
    #                 magic_bytes = f.read(3)
    #                 valid_bed = magic_bytes == b'\x6c\x1b\x01'
                    
    #             print(f"- 유효한 PLINK .bed 형식: {'예' if valid_bed else '아니오'}")
    #             print(f"- 파일 크기: {os.path.getsize(bed_path) / (1024*1024):.2f} MB")
            
    #         except Exception as e:
    #             print(f".bed 파일 pandas-plink 분석 중 오류 발생: {e}")
                
    #     except Exception as e:
    #         print(f".bed 파일 분석 중 오류 발생: {e}")

def explore_demographic_data(demo_file):
    """
    역학 데이터(total_demo.txt)의 구조를 탐색합니다.
    
    Args:
        demo_file: 역학 데이터 파일 경로
    """
    print("\n" + "="*80)
    print("역학 데이터 탐색 (total_demo.txt)")
    print("="*80)
    
    # 1. 파일 존재 확인
    if not os.path.exists(demo_file):
        print(f"오류: 역학 데이터 파일을 찾을 수 없습니다: {demo_file}")
        return
    
    file_size = os.path.getsize(demo_file) / (1024*1024)
    print(f"파일 크기: {file_size:.2f} MB")
    
    # 2. 파일 형식 감지
    try:
        # 첫 몇 줄을 읽어 구분자 추측
        with open(demo_file, 'r', encoding='utf-8') as f:
            first_lines = [next(f) for _ in range(5)]
        
        # 구분자 감지
        if '\t' in first_lines[0]:
            sep = '\t'
            print("구분자: 탭 (\\t)")
        elif ',' in first_lines[0]:
            sep = ','
            print("구분자: 쉼표 (,)")
        else:
            sep = None
            print("구분자를 감지할 수 없습니다. 공백으로 시도합니다.")
        
        # 데이터 로드
        if sep:
            df = pd.read_csv(demo_file, sep=sep, nrows=1000)  # 처음 1000행만 로드
        else:
            df = pd.read_csv(demo_file, delim_whitespace=True, nrows=1000)
        
        # 3. 데이터 기본 정보
        print(f"\n- 샘플 크기: {len(df)} 행 x {len(df.columns)} 열")
        print(f"- 열 이름 (처음 20개): {df.columns[:20].tolist()}")
        
        # 4. 데이터 요약
        print("\n데이터 요약 (처음 5개 행):")
        print(df.head())
        
        # 5. 데이터 유형 요약
        print("\n데이터 유형 요약:")
        dtypes_summary = df.dtypes.value_counts()
        for dtype, count in dtypes_summary.items():
            print(f"- {dtype}: {count}개 열")
        
        # 6. 결측값 분석
        missing_data = df.isnull().sum()
        print("\n결측값 상위 10개 열:")
        print(missing_data.sort_values(ascending=False).head(10))

        # 모든 데이터가 결측값인 열 확인
        all_missing_cols = df.columns[df.isnull().all()]
        if all_missing_cols.any():
            print("\n모든 데이터가 결측값인 열:")
            print(all_missing_cols)
        print(f"모든 데이터가 결측값인 열의 수: {len(all_missing_cols)} / {len(df.columns)}")
        
        # 7. 인구통계학적 변수 확인 (있는 경우)
        demographic_vars = ['age', 'sex', 'gender', 'ethnicity', 'race']
        found_vars = [col for col in df.columns if any(var in col.lower() for var in demographic_vars)]
        
        if found_vars:
            print("\n발견된 인구통계학적 변수:")
            for var in found_vars:
                print(f"- {var}")
                if df[var].dtype == 'object' or df[var].dtype == 'category':
                    print(f"  범주: {df[var].value_counts().head(5)}")
                else:
                    print(f"  요약: {df[var].describe().round(2)}")
        
        # 8. 질병 코드 변수 확인 (ICD 코드)
        icd_vars = [col for col in df.columns if 'icd' in col.lower() or 'disease' in col.lower() or 'diagnosis' in col.lower()]
        
        if icd_vars:
            print("\n발견된 질병 관련 변수:")
            for var in icd_vars[:10]:  # 처음 10개만 표시
                print(f"- {var}")
                if df[var].dtype == 'object' or df[var].dtype == 'category':
                    print(f"  가장 흔한 값: {df[var].value_counts().head(3)}")

        # 모든 열에 대해 데이터 타입 확인
        print("\n모든 열에 대해 데이터 타입 확인:")
        for col in df.columns:
            print(f"{col}: {df[col].dtype}")
        
    except Exception as e:
        print(f"역학 데이터 분석 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

<>:31: SyntaxWarning: invalid escape sequence '\s'
<>:56: SyntaxWarning: invalid escape sequence '\s'
<>:31: SyntaxWarning: invalid escape sequence '\s'
<>:56: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_3275933/2419886732.py:31: SyntaxWarning: invalid escape sequence '\s'
  fam_df = pd.read_csv(fam_path, sep='\s+', header=None,
/tmp/ipykernel_3275933/2419886732.py:56: SyntaxWarning: invalid escape sequence '\s'
  bim_df = pd.read_csv(bim_path, sep='\s+', header=None,


In [4]:
GENOME_DATA_PATH = "/home/soyeon/workspace/data/GENOME/UKB_genome"  # 확장자 없이
DEMO_DATA_PATH = "/home/soyeon/workspace/data/GENOME/total_demo.txt"

In [8]:
# 유전체 데이터 탐색
explore_genetic_data(GENOME_DATA_PATH)


유전체 데이터 탐색 (.bed, .fam, .bim)
/home/soyeon/workspace/data/GENOME/UKB_genome.bed - 존재함 (크기: 91317.88 MB)
/home/soyeon/workspace/data/GENOME/UKB_genome.fam - 존재함 (크기: 11.64 MB)
/home/soyeon/workspace/data/GENOME/UKB_genome.bim - 존재함 (크기: 21.47 MB)

.fam 파일 구조 (샘플 정보):
- 샘플 수: 488377
- 열 목록: ['FID', 'IID', 'PID', 'MID', 'Sex', 'Phenotype']
- 처음 5개 행:
   FID  IID  PID  MID  Sex  Phenotype
0   -1   -1    0    0    0         -9
1   -2   -2    0    0    0         -9
2   -3   -3    0    0    0         -9
3   -4   -4    0    0    0         -9
4   -5   -5    0    0    0         -9

- 성별 분포:
  여성 (코드 2): 264717명 (54.20%)
  남성 (코드 1): 223411명 (45.75%)
  알 수 없음 (코드 0): 249명 (0.05%)

.bim 파일 구조 (SNP 정보):
- SNP 수: 784256
- 열 목록: ['CHR', 'SNP', 'CM', 'POS', 'A1', 'A2']
- 처음 5개 행:
   CHR          SNP  CM     POS A1 A2
0    1   rs28659788   0  723307  G  C
1    1  rs116587930   0  727841  A  G
2    1  rs116720794   0  729632  T  C
3    1    rs3131972   0  752721  A  G
4    1   rs12184325   0  754105  T

In [5]:
# 역학 데이터 탐색
explore_demographic_data(DEMO_DATA_PATH)


역학 데이터 탐색 (total_demo.txt)
파일 크기: 38006.03 MB
구분자: 탭 (\t)


/tmp/ipykernel_3275933/2419886732.py:197: DtypeWarning: Columns (5,7,1399,1400,1401,2935,2936,2937,2940,2941,2942,2943,3007,3008,3009,3012,3013,3014,3015,3074,3075,3076,3077,3078,3079,3080,3085,3086,3087,3088,3089,3090,3091,3468,3470,3974,3975,3976,3977,3978,3979,3980,3981,3982,3983,3984,3985,3986,3987,3988,4004,4005,4006,4007,4008,4009,4010,4011,4012,4013,4014,4015,4016,4017,4018,4574,4575,4576,4577,4578,4579,4580,4581,4582,4583,4584,4585,4586,4587,4588,4604,4605,4606,4607,4608,4609,4610,4611,4612,4613,4614,4615,4616,4617,4618,5127,5128,5129,5130,5131,5132,5133,5134,5135,5136,5137,5138,5167,5168,5169,5170,5171,5175,5176,5177,5178,5179,5180,5181,5182,5183,5184,5185,5432,5434,5444,5446,5468,5470,5747,5748,5749,5756,5757,5758,5765,5766,5767,5772,5773,5774,6115,6116,6117,6127,6128,6129,6133,6134,6135,6140,6141,6145,6146,6147,6148,6149,6150,6151,6152,6163,6164,6165,6170,6171,6172,6173,6175,6176,6177,6178,6179,6181,6182,6183,6187,6188,6189,6193,6194,6195,6199,6200,6201,6205,6206,6207,6208,6


- 샘플 크기: 1000 행 x 24094 열
- 열 이름 (처음 20개): ['eid', 'X31.0.0', 'X34.0.0', 'X52.0.0', 'X53.0.0', 'X53.1.0', 'X53.2.0', 'X53.3.0', 'X55.0.0', 'X55.1.0', 'X55.2.0', 'X55.3.0', 'X670.0.0', 'X670.1.0', 'X670.2.0', 'X670.3.0', 'X680.0.0', 'X680.1.0', 'X680.2.0', 'X680.3.0']

데이터 요약 (처음 5개 행):
       eid  X31.0.0  X34.0.0  X52.0.0     X53.0.0 X53.1.0 X53.2.0 X53.3.0  \
0  1000015        0     1945        5  2008-11-27     NaN     NaN     NaN   
1  1000027        1     1966        3  2010-05-21     NaN     NaN     NaN   
2  1000039        0     1954        4  2009-07-04     NaN     NaN     NaN   
3  1000040        0     1964        7  2008-09-11     NaN     NaN     NaN   
4  1000053        1     1960        3  2008-01-11     NaN     NaN     NaN   

   X55.0.0  X55.1.0  ...  X132596.0.0  X132597.0.0  X132598.0.0  X132599.0.0  \
0       11      NaN  ...          NaN          NaN          NaN          NaN   
1        5      NaN  ...          NaN          NaN          NaN          NaN   
2        